In [1]:
!pip install "numpy<1.24.0"

In [11]:
import numpy
print(numpy.__version__)

1.23.5


In [15]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import random as rnd

from scipy.sparse import *

In [13]:
urm_path = './content/data_train.csv'
urm_all_df = pd.read_csv(filepath_or_buffer=urm_path,
                                sep=",",
                                header=0,
                                dtype={0:int, 1:int, 2:float},
                                engine='python')

urm_all_df.columns = ["UserID", "ItemID", "Interaction"]
print ("The number of interactions is {}".format(len(urm_all_df)))

The number of interactions is 478730


In [5]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.8)

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout

cutoff_list=[5, 10, 15]

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

EvaluatorHoldout: Ignoring 3040 (23.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2542 (19.5%) Users that have less than 1 test interactions


# Hybrid of models with the same structure

The underlying idea is that if you have some models with the same underlying structure, you can merge them in a single one. The result is to define a function that give the various model's parameters creates a new model with its own parameters.
In this example we will see how to combine different item-item similarity models in a new one.

In particular: `ItemKNNCFRecommender` and `P3AlphaRecommender`. Both use and learn a similarity matrix to provide recommendations. Notice that here we are merging a KNN-based with a Graph-based model.


Let's start using our best model yet

SLIM elasticnet with removed empty profiles and the hyperparameters: 
alpha = 0.0015746723778813712, l1_ratio = 0.005,topK = 100, max_iter=100, tol=1e-4,positive_only=True. On validation it had MAP@10 on val is 0.08221

In [7]:
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Recommenders.Similarity.Compute_Similarity_Python import Incremental_Similarity_Builder
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import sys
import time


class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available

    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.

        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)
        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        similarity_builder = Incremental_Similarity_Builder(self.n_items, initial_data_block=self.n_items*self.topK, dtype = np.float32)

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values
            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            # Check if there are more data points than topK, if so, extract the set of K best values
            if len(nonzero_model_coef_value) > self.topK:
                # Partition the data because this operation does not require to fully sort the data
                relevant_items_partition = np.argpartition(-np.abs(nonzero_model_coef_value), self.topK-1, axis=0)[0:self.topK]
                nonzero_model_coef_index = nonzero_model_coef_index[relevant_items_partition]
                nonzero_model_coef_value = nonzero_model_coef_value[relevant_items_partition]

            similarity_builder.add_data_lists(row_list_to_add=nonzero_model_coef_index,
                                              col_list_to_add=np.ones(len(nonzero_model_coef_index), dtype = int) * currentItem,
                                              data_list_to_add=nonzero_model_coef_value)


            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        self.W_sparse = similarity_builder.get_SparseMatrix()

In [8]:
slim_elastic_recommender = SLIMElasticNetRecommender(URM_all)
slim_elastic_recommender.fit(alpha = 0.0015746723778813712, l1_ratio = 0.005,topK = 100)

SLIMElasticNetRecommender: URM Detected 387 ( 3.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 126 ( 0.6%) items with no interactions.


d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(


SLIMElasticNetRecommender: Processed 3998 (17.9%) in 5.00 min. Items per second: 13.32


d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.

SLIMElasticNetRecommender: Processed 8389 (37.5%) in 10.00 min. Items per second: 13.98


...

SLIMElasticNetRecommender: Processed 13960 (62.5%) in 15.00 min. Items per second: 15.51


d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.

SLIMElasticNetRecommender: Processed 18760 (83.9%) in 20.00 min. Items per second: 15.63


d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.sparse_enet_coordinate_descent(
d:\Anaconda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:589: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.

SLIMElasticNetRecommender: Processed 22348 (100.0%) in 23.66 min. Items per second: 15.74


In [9]:
slim_elastic_recommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2197697 stored elements in Compressed Sparse Row format>

In [10]:
from scipy.sparse import save_npz
save_npz("./content/item_item_similarity/slim_elastic_complete.npz", slim_elastic_recommender.W_sparse)

---

# IBCF 

Item-base CF where hyperparamters were obtained with 10-fold CV.
topK = 13,shrink=10, silimarity = 'cosine'

In [11]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [12]:
IBCFrecommender = ItemKNNCFRecommender(URM_all)
IBCFrecommender.fit(topK=14,shrink=7)

ItemKNNCFRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 126 ( 0.6%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 22348 (100.0%), 1383.93 column/sec. Elapsed time 16.15 sec


In [13]:
save_npz("./content/item_item_similarity/IBCF_complete.npz", IBCFrecommender.W_sparse)

---

# Evaluation

In [22]:
def precision(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score

def recall(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score

def AP(recommended_items, relevant_items):

    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_model(URM_test, recommender_object, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0


    for user_id in range(URM_test.shape[0]):

        relevant_items = URM_test.indices[URM_test.indptr[user_id]:URM_test.indptr[user_id+1]]

        if len(relevant_items)>0:

            recommended_items = recommender_object.recommend(user_id, cutoff=at,remove_seen_flag=True)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    mean_precision = cumulative_precision / num_eval
    mean_recall = cumulative_recall / num_eval
    MAP = cumulative_AP / num_eval

    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP))
    return MAP, mean_precision, mean_recall

In [12]:
evaluate_model(URM_test,slim_elastic_recommender)

Recommender results are: Precision = 806.0000, Recall = 1132.6422, MAP = 0.0653


(0.06532939073507661, 0.0770186335403768, 0.10823145921793417)

In [23]:
evaluate_model(URM_test,IBCFrecommender)

Recommender results are: Precision = 759.5000, Recall = 1128.5813, MAP = 0.0607


(0.06070750121552377, 0.07257525083612454, 0.1078434143433888)

---

Now that we have both models fit with `URM_train` how can we merge them?


Well, we could take the advantage that they have the same underlying structure, i.e. a similarity matrix

In [24]:
slim_elastic_recommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2085811 stored elements in Compressed Sparse Row format>

In [25]:
IBCFrecommender.W_sparse

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 305280 stored elements in Compressed Sparse Row format>

When we create hybrids we always have to account for:
* A weight, which is a hyperparameter to tune. 
* What is the range and distribution of the values we are merging

Imagine that the values of one similarity are orders of magnitude higher than the other, then the smaller values would get absorbed by the bigger ones. 

In [44]:
alpha = 0.2
new_similarity = (1 - alpha) * slim_elastic_recommender.W_sparse + alpha * IBCFrecommender.W_sparse
new_similarity

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2115592 stored elements in Compressed Sparse Row format>

Moreover, we can apply the same optimizations to this similarity as we applied to the others (for example: topK neighbors)

The framework includes a recommender class which supports a custom similarity

In [45]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)

ItemKNNCustomSimilarityRecommender: URM Detected 836 ( 6.4%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 476 ( 2.1%) items with no interactions.


In [46]:
evaluate_model(URM_test,recommender_object)

Recommender results are: Precision = 833.3000, Recall = 1220.9303, MAP = 0.0677


(0.06770425768651123, 0.07962732919255107, 0.11666796467752119)

# Find the model using our best 4 submissions

In [1]:
from scipy.sparse import load_npz

Best submission is SLIM_elastic:

In [2]:
SLIM_elastic_sim = load_npz("./content/item_item_similarity/slim_elastic_complete.npz")

In [3]:
SLIM_elastic_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 2197697 stored elements in Compressed Sparse Row format>

Then we have:

In [4]:
easer_sim = load_npz("./content/item_item_similarity/easer_complete.npz")

In [5]:
IBCF_sim = load_npz("./content/item_item_similarity/IBCF_complete.npz")

In [6]:
IBCF_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 310959 stored elements in Compressed Sparse Row format>

In [8]:
rp3beta_sim = load_npz("./content/item_item_similarity/rp3beta_complete.npz")

In [9]:
rp3beta_sim

<22348x22348 sparse matrix of type '<class 'numpy.float32'>'
	with 3619038 stored elements in Compressed Sparse Row format>

## Trying merging them

In [17]:
URM_all = sps.coo_matrix((urm_all_df["Interaction"].values,
                          (urm_all_df["UserID"].values, urm_all_df["ItemID"].values)))

In [18]:
URM_all = URM_all.tocsr()

In [19]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

Trying with SLIM and IBCF

In [23]:
alpha = 0
new_similarity = (1 - alpha) * SLIM_elastic_sim + alpha * IBCF_sim
recommender_object = ItemKNNCustomSimilarityRecommender(URM_all)
recommender_object.fit(new_similarity)
evaluate_model(URM_all,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 387 ( 3.0%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 126 ( 0.6%) items with no interactions.
Recommender results are: Precision = 0.0000, Recall = 0.0000, MAP = 0.0000


(0.0, 0.0, 0.0)

In [30]:
alpha = 0.2
new_similarity = (1 - alpha) * SLIM_elastic_sim + alpha * IBCF_sim
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_train,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 823 ( 6.3%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 445 ( 2.0%) items with no interactions.
Recommender results are: Precision = 1481.3000, Recall = 2222.0025, MAP = 0.1250


(0.12501269044950036, 0.14130496995134745, 0.2119624604636375)

In [32]:
alpha = 0.7
new_similarity = (1 - alpha) * SLIM_elastic_sim + alpha * IBCF_sim
recommender_object = ItemKNNCustomSimilarityRecommender(URM_train)
recommender_object.fit(new_similarity)
evaluate_model(URM_train,recommender_object)

ItemKNNCustomSimilarityRecommender: URM Detected 823 ( 6.3%) users with no interactions.
ItemKNNCustomSimilarityRecommender: URM Detected 445 ( 2.0%) items with no interactions.
Recommender results are: Precision = 1107.5000, Recall = 1607.6633, MAP = 0.0926


(0.09261699817557226, 0.10564723838596149, 0.1533590838265386)